In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import requests
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import date

In [7]:
def print_news(tickers):
    """
    Print stock news headlines from finviz.com for selected tickers
    
    Parameters
    ----------
    tickers : list
        A list of tickers
    Returns
    -------
        a dataframe of news headers for each ticker
    Examples
    --------
    >>> print_news(['AAPL', 'TSLA', 'AMZN'])
    """
    
    # Get Data from finviz
    finviz_url = 'https://finviz.com/quote.ashx?t='
    news_tables = {}

    for ticker in tickers:
        url = finviz_url + ticker
        req = Request(url=url, headers={'user-agent': 'my-app/0.0.1'}) 
        resp = urlopen(req)    
        html = BeautifulSoup(resp, "html.parser")
        news_table = html.find(id='news-table')
        news_tables[ticker] = news_table
    # save first n news headers in a data frame
    headers_df = {
        'Ticker': [],
        'Headers': [],
        'Time and Date': []
    }
    try:
        for ticker in tickers:
            df = news_tables[ticker]
            df_tr = df.findAll('tr')
            for i, table_row in enumerate(df_tr):
                headers_df['Ticker'].append(ticker)
                headers_df['Headers'].append(table_row.a.text)
                td_text = table_row.td.text
                headers_df['Time and Date'].append(td_text.strip())
        headers_df = pd.DataFrame(headers_df)
        foo = lambda x: pd.Series([i for i in reversed(x.split(' '))])
        headers_df[['Time', 'Date']] = headers_df['Time and Date'].apply(foo)
        headers_df = headers_df.fillna(method='ffill')
        headers_df = headers_df[['Ticker', 'Headers', 'Date', 'Time']]
    except KeyError:
        pass
    return headers_df

In [8]:
def sentiment_scores(headers_df):
    """
    conduct sentiment analysis and score each line of headers for selected tickers
    Parameters
    ----------
    headers_df : dataframe
        a data frame containing ticker, headers, time and date
    Returns
    -------
        a dataframe of sentiment scores for news headers for each ticker
    Examples
    --------
    >>> sentiment_scores(headers_df)
    """
    
    analyzer = SentimentIntensityAnalyzer()
    scores = headers_df['Headers'].apply(analyzer.polarity_scores).tolist()
    df_scores = pd.DataFrame(scores)
    df_scores = df_scores.rename(columns={'neg': 'Negative', 'neu': 'Neutral', 'pos': 'Positive', 'compound': 'Compound'})
    score_df = pd.concat([headers_df, df_scores], axis=1)
    return score_df

In [9]:
def average_score(score_df):
    """
    show average sentiment scores of each ticker
    
    Parameters
    ----------
    headers_df : dataframe
        a data frame containing ticker, headers, time, date, sentiment scores
    Returns
    -------
        a dataframe of average sentiment scores for news headers for each ticker
    Examples
    --------
    >>> average_score(score_df)
    """
    
    return score_df.groupby('Ticker').mean().round(2).sort_values('Compound', ascending=False)


In [10]:
tickers = ['AAPL', 'TSLA', 'AMZN']
a = print_news(tickers)
b = sentiment_scores(a)
c = average_score(b)

In [11]:
a

,Ticker,Headers,Date,Time
0,AAPL,Chinas Growing Outbreak; U.S. Curbs May Ease: ...,Feb-23-22,09:26PM
1,AAPL,Top Millennial And Gen Z Stock Picks From Tesl...,Feb-23-22,04:54PM
2,AAPL,Cloudflare to Buy Area 1 Security in Push to P...,Feb-23-22,04:30PM
3,AAPL,Is Now the Right Time to Buy Big Tech Stocks?,Feb-23-22,03:15PM
4,AAPL,Spotify Gambles on Cars to Outplay Apple,Feb-23-22,01:01PM
...,...,...,...,...
295,AMZN,Market strategist: Defensive trades can roll o...,Feb-17-22,03:41PM
296,AMZN,Amazons data center arm buys three Sterling of...,Feb-17-22,03:18PM
297,AMZN,10 Best Index Funds to Invest In 2022,Feb-17-22,03:15PM
298,AMZN,Amazons Outlook Is Bright. Dan Loeb Is Buying ...,Feb-17-22,01:38PM


In [12]:
b

,Ticker,Headers,Date,Time,Negative,Neutral,Positive,Compound
0,AAPL,Chinas Growing Outbreak; U.S. Curbs May Ease: ...,Feb-23-22,09:26PM,0.000,0.625,0.375,0.4939
1,AAPL,Top Millennial And Gen Z Stock Picks From Tesl...,Feb-23-22,04:54PM,0.000,0.886,0.114,0.2023
2,AAPL,Cloudflare to Buy Area 1 Security in Push to P...,Feb-23-22,04:30PM,0.000,0.667,0.333,0.6124
3,AAPL,Is Now the Right Time to Buy Big Tech Stocks?,Feb-23-22,03:15PM,0.000,1.000,0.000,0.0000
4,AAPL,Spotify Gambles on Cars to Outplay Apple,Feb-23-22,01:01PM,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
295,AMZN,Market strategist: Defensive trades can roll o...,Feb-17-22,03:41PM,0.000,0.916,0.084,0.0258
296,AMZN,Amazons data center arm buys three Sterling of...,Feb-17-22,03:18PM,0.121,0.879,0.000,-0.0258
297,AMZN,10 Best Index Funds to Invest In 2022,Feb-17-22,03:15PM,0.000,0.625,0.375,0.6369
298,AMZN,Amazons Outlook Is Bright. Dan Loeb Is Buying ...,Feb-17-22,01:38PM,0.079,0.714,0.207,0.4215


In [13]:
c

,Negative,Neutral,Positive,Compound
Ticker,,,,
AMZN,0.06,0.80,0.14,0.12
AAPL,0.06,0.82,0.12,0.08
TSLA,0.07,0.83,0.10,0.04
